In [ ]:
import mysql.connector
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from credentials import ipCred, usernameCred, passwordCred, databaseCred
# Import Keras libraries for building the LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def train_lstm_model(ticker):
    # 1. Database connection parameters and query
    db_config = {
        'host': ipCred,
        'user': usernameCred,
        'password': passwordCred,
        'database': databaseCred
    }
    
    query = f"""
        SELECT Date_ AS date, Close_ AS close
        FROM {ticker}_stock_data
        WHERE Date_ BETWEEN '2020-01-01' AND '2025-01-01'
        ORDER BY Date_ ASC;
    """
    
    # Connect to the database and load data into a DataFrame
    conn = mysql.connector.connect(**db_config)
    df = pd.read_sql(query, conn)
    conn.close()
    
    # 2. Preprocess the data
    # Convert date column to datetime and set it as index
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Keep only the 'close' price column
    df = df[['close']]
    
    # Scale the data to the range [0,1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df.values)
    
    # 3. Create training sequences (using a look-back window)
    look_back = 60  # e.g., use 60 days of data to predict the next day
    X, y = [], []
    
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i - look_back:i, 0])
        y.append(scaled_data[i, 0])
    
    X, y = np.array(X), np.array(y)
    # Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # 4. Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # 5. Train the model
    model.fit(X, y, epochs=20, batch_size=32)
    
    # 6. Save the trained model and preprocessing objects
    # Saving the model using Keras' built-in save function (recommended for deep learning models)
    model.save(f'{ticker}_lstm_model.h5')
    
    # Save the scaler and look-back period with pickle for later use in prediction
    with open(f'{ticker}_scaler.pkl', 'wb') as scaler_file:
        pickle.dump(scaler, scaler_file)
    
    with open(f'{ticker}_look_back.pkl', 'wb') as lb_file:
        pickle.dump(look_back, lb_file)
    
    print(f"Model and preprocessing objects for {ticker} have been saved.")

# Example usage:
if __name__ == "__main__":
    ticker_symbol = input("Enter the ticker symbol (e.g., AAPL, ORCL): ").strip().upper()
    train_lstm_model(ticker_symbol)

/var/folders/v1/gw8x5_gj64gfk28hkxlbrzr00000gn/T/ipykernel_87030/3521170215.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Epoch 1/20


2025-03-08 17:57:47.871683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-03-08 17:57:47.872626: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-03-08 17:57:47.873100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

38/38 [==============================] - 2s 14ms/step - loss: 0.0445
Epoch 2/20
38/38 [==============================] - 1s 14ms/step - loss: 0.0020
Epoch 3/20
38/38 [==============================] - 1s 15ms/step - loss: 9.1001e-04
Epoch 4/20
38/38 [==============================] - 1s 15ms/step - loss: 8.8414e-04
Epoch 5/20
38/38 [==============================] - 1s 15ms/step - loss: 8.6525e-04
Epoch 6/20
38/38 [==============================] - 1s 15ms/step - loss: 8.5911e-04
Epoch 7/20
38/38 [==============================] - 1s 15ms/step - loss: 8.6170e-04
Epoch 8/20
38/38 [==============================] - 1s 15ms/step - loss: 8.0280e-04
Epoch 9/20
38/38 [==============================] - 1s 15ms/step - loss: 7.9115e-04
Epoch 10/20
38/38 [==============================] - 1s 15ms/step - loss: 8.7210e-04
Epoch 11/20
38/38 [==============================] - 1s 14ms/step - loss: 7.7850e-04
Epoch 12/20
38/38 [==============================] - 1s 15ms/step - loss: 8.2242e-04
Epoch 13